In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

In [ ]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [ ]:
# Normalize + Flatten
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

In [ ]:
x_train = x_train.reshape((len(x_train), 784))
x_test  = x_test.reshape((len(x_test), 784))

In [ ]:
# Train only on digit '3' (normal class)
train_normal = x_train[y_train == 3]
test_normal = x_test[y_test == 3]

In [ ]:
# build autoencoder

# Encoder converts input to latent representation
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)     # latent size 32

In [ ]:
# d) Decoder converts latent rep. back to original input
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [ ]:
# autoencoder model
autoencoder = Model(input_img, decoded)

In [ ]:
# e) Compile model with Optimizer, Loss & Metrics
autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.summary()


In [ ]:
# Train model
autoencoder.fit(train_normal, train_normal,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(test_normal,test_normal))

In [ ]:
# ---- Anomaly Detection ----
normal_test = x_test[y_test==3]
anomaly_test = x_test[y_test==7]

In [ ]:
test_data = np.concatenate([normal_test[:100], anomaly_test[:100]])
test_labels = np.array([0]*100 + [1]*100)


In [ ]:
# reconstruction
recon = autoencoder.predict(test_data)
errors = np.mean(np.abs(test_data - recon),axis=1)

In [ ]:
# threshold
threshold = np.mean(errors[test_labels==0]) + 2*np.std(errors[test_labels==0])
print("Threshold =", threshold)
print("Anomalies Detected =", np.sum(errors > threshold))
print("Total Samples =", len(test_data))